In [1]:
PROJECT_ID = "qp-acc-aia-migrate-2021-06" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [1]:
BUCKET_NAME = "hassan-ai-platform-temp-test" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}

In [2]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Creating gs://hassan-ai-platform-temp-test/...


In [3]:
! gsutil ls -al gs://$BUCKET_NAME

In [5]:
! pip install numpy>=1.16.0 scikit-learn==0.20.2

In [11]:
# !mkdir custom_routione_prediction
%cd custom_routione_prediction

/home/jupyter/custom_routione_prediction


In [12]:
%%writefile preprocess.py
import numpy as np

class MySimpleScaler(object):
  def __init__(self):
    self._means = None
    self._stds = None

  def preprocess(self, data):
    if self._means is None: # during training only
      self._means = np.mean(data, axis=0)

    if self._stds is None: # during training only
      self._stds = np.std(data, axis=0)
      if not self._stds.all():
        raise ValueError('At least one column has standard deviation of 0.')

    return (data - self._means) / self._stds

Overwriting preprocess.py


In [13]:
import pickle

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

from preprocess import MySimpleScaler

iris = load_iris()
scaler = MySimpleScaler()
X = scaler.preprocess(iris.data)
y = iris.target

model = RandomForestClassifier()
model.fit(X, y)

joblib.dump(model, 'model.joblib')
with open ('preprocessor.pkl', 'wb') as f:
  pickle.dump(scaler, f)

/opt/conda/lib/python3.7/site-packages/sklearn/datasets/base.py:242: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  target = np.empty((n_samples,), dtype=np.int)
/opt/conda/lib/python3.7/site-packages/sklearn/datasets/base.py:246: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check th

In [14]:
%%writefile predictor.py
import os
import pickle

import numpy as np
from sklearn.datasets import load_iris
from sklearn.externals import joblib

class MyPredictor(object):
  def __init__(self, model, preprocessor):
    self._model = model
    self._preprocessor = preprocessor
    self._class_names = load_iris().target_names

  def predict(self, instances, **kwargs):
    inputs = np.asarray(instances)
    preprocessed_inputs = self._preprocessor.preprocess(inputs)
    if kwargs.get('probabilities'):
      probabilities = self._model.predict_proba(preprocessed_inputs)
      return probabilities.tolist()
    else:
      outputs = self._model.predict(preprocessed_inputs)
      return [self._class_names[class_num] for class_num in outputs]

  @classmethod
  def from_path(cls, model_dir):
    model_path = os.path.join(model_dir, 'model.joblib')
    model = joblib.load(model_path)

    preprocessor_path = os.path.join(model_dir, 'preprocessor.pkl')
    with open(preprocessor_path, 'rb') as f:
      preprocessor = pickle.load(f)

    return cls(model, preprocessor)

Overwriting predictor.py


In [15]:
%%writefile setup.py
from setuptools import setup

setup(
    name='my_custom_code',
    version='0.1',
    scripts=['predictor.py', 'preprocess.py'])

Writing setup.py


In [16]:
! python setup.py sdist --formats=gztar

running sdist
running egg_info
creating my_custom_code.egg-info
writing my_custom_code.egg-info/PKG-INFO
writing dependency_links to my_custom_code.egg-info/dependency_links.txt
writing top-level names to my_custom_code.egg-info/top_level.txt
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'
reading manifest file 'my_custom_code.egg-info/SOURCES.txt'
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'

running check


creating my_custom_code-0.1
creating my_custom_code-0.1/my_custom_code.egg-info
copying files to my_custom_code-0.1...
copying predictor.py -> my_custom_code-0.1
copying preprocess.py -> my_custom_code-0.1
copying setup.py -> my_custom_code-0.1
copying my_custom_code.egg-info/PKG-INFO -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/SOURCES.txt -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/dependency_links.txt -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/top_lev

In [17]:
! gsutil cp ./dist/my_custom_code-0.1.tar.gz gs://$BUCKET_NAME/custom_prediction_routine_tutorial/my_custom_code-0.1.tar.gz
! gsutil cp model.joblib preprocessor.pkl gs://$BUCKET_NAME/custom_prediction_routine_tutorial/model

Copying file://./dist/my_custom_code-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.2 KiB/  1.2 KiB]                                                
Operation completed over 1 objects/1.2 KiB.                                      
Copying file://model.joblib [Content-Type=application/octet-stream]...
Copying file://preprocessor.pkl [Content-Type=application/octet-stream]...      
/ [2 files][ 21.7 KiB/ 21.7 KiB]                                                
Operation completed over 2 objects/21.7 KiB.                                     


In [29]:
MODEL_NAME = 'IrisPredictor4'
REGION = 'global'

In [31]:
! gcloud ai-platform models create $MODEL_NAME \
  --region $REGION

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) INVALID_ARGUMENT: Field: model.regions Error: The provided GCE region 'global' is not available, or your project needs to be whitelisted to use it. Other available regions are [asia-northeast1, europe-west1, us-central1, us-east1, us-east4].
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: The provided GCE region 'global' is not available, or your project
      needs to be whitelisted to use it. Other available regions are [asia-northeast1,
      europe-west1, us-central1, us-east1, us-east4].
    field: model.regions


In [32]:
MODEL_DIR="gs://hassan-ai-platform-temp-test/custom_prediction_routine_tutorial/model/ "
VERSION_NAME = 'version1'
MODEL_NAME = 'IrisPredictor4'
CUSTOM_CODE_PATH="gs://hassan-ai-platform-temp-test/custom_prediction_routine_tutorial/my_custom_code-0.1.tar.gz"
PREDICTOR_CLASS=" predictor.MyPredictor"
REGION = "global"
MACHINE_TYPE ="mls1-c4-m2"

In [33]:
print(CUSTOM_CODE_PATH)

gs://hassan-ai-platform-temp-test/custom_prediction_routine_tutorial/my_custom_code-0.1.tar.gz


In [34]:
# --quiet automatically installs the beta component if it isn't already installed 
! gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --region $REGION \
  --runtime-version=1.13 \
  --python-version=3.5 \
  --origin $MODEL_DIR \
  --package-uris $CUSTOM_CODE_PATH \
  --prediction-class $PREDICTOR_CLASS \
  --machine-type $MACHINE_TYPE

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [35]:
! pip install --upgrade google-api-python-client

     |████████████████████████████████| 7.4 MB 7.3 MB/s eta 0:00:01
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.18.0
    Uninstalling google-api-python-client-2.18.0:
      Successfully uninstalled google-api-python-client-2.18.0


In [37]:
import googleapiclient.discovery
PROJECT_ID = "qp-acc-aia-migrate-2021-06"
instances = [
  [6.7, 3.1, 4.7, 1.5],
  [4.6, 3.1, 1.5, 0.2],
]

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

['versicolor', 'setosa']


In [38]:
#Sending Keyword Arguments
response = service.projects().predict(
    name=name,
    body={'instances': instances, 'probabilities': True}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

[[0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]


In [ ]:
# Delete version resource
! gcloud ai-platform versions delete $VERSION_NAME --quiet --model $MODEL_NAME 

# Delete model resource
! gcloud ai-platform models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r gs://$BUCKET_NAME/custom_prediction_routine_tutorial